In [56]:
import os
import pandas as pd 
import plotly.express as px 
import plotly.graph_objects as go 
import numpy as np 
from datetime import datetime as dt 
import json

import dash
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dash_daq as daq

# external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/slate/bootstrap.min.css']

app = dash.Dash(__name__,
     meta_tags=[{'name': 'viewport',
                 'content': 'width=device-width, initial-scale=1.0'}]

)
server = app.server

df = pd.read_csv('Apple Music Activity v2.csv')
df.dropna(thresh = 2)

fdf = df.groupby(['Apple Music Subscription', 'Artist Name', 'Content Name', 'Content Provider', 'Content Specific Type', 'End Reason Type', 'Event Start Timestamp', 'Feature Name', 'Genre'])['Store Country Name'].count().reset_index()
fdf = fdf.replace("`|’", "'", regex=True)
fdf = fdf.rename(columns = {'Store Country Name':'Play Count','Artist Name':'Artist','Content Name':'Song'})
fdf.loc[fdf['Genre'].str.contains('Metal'), 'Genre'] = 'Metal'
fdf.loc[fdf['Genre'].str.contains('Rock'), 'Genre'] = 'Rock'
fdf.loc[fdf['Genre'].str.contains('Rap'), 'Genre'] = 'Hip Hop/Rap'
fdf.loc[fdf['Genre'].str.contains('Hip'), 'Genre'] = 'Hip Hop/Rap'
fdf.loc[fdf['Genre'].str.contains('Country'), 'Genre'] = 'Country'
fdf.loc[fdf['Genre'].str.contains('Alternative'), '"genre'] = 'Alternative'
fdf.loc[fdf['Genre'].str.contains('R&B'), 'Genre'] = 'R&B/Soul'
fdf.loc[fdf['Genre'].str.contains('Pop'), 'Genre'] = 'Pop'
fdf = fdf[fdf['Apple Music Subscription'] == True]

fdf['Date'] = pd.to_datetime(fdf['Event Start Timestamp'])
fdf['Year'] = fdf['Date'].apply(lambda datetime: datetime.year)
fdf['Month'] = fdf['Date'].apply(lambda datetime: datetime.month)
fdf['Weekday'] = fdf['Date'].apply(lambda datetime: datetime.weekday())
fdf['Time'] = fdf['Event Start Timestamp'].str.split('T').str[-1]
fdf['Time'] = fdf['Time'].str[:5]

weekdayn = []

for n in fdf['Weekday']:
    if n == 6:
        weekdayn.append('Sunday')
    elif n == 0:
        weekdayn.append('Monday')
    elif n == 1:
        weekdayn.append('Tuesday')
    elif n == 2:
        weekdayn.append('Wednesday')
    elif n == 3:
        weekdayn.append('Thursday')
    elif n == 4:
        weekdayn.append('Friday')
    else:
        weekdayn.append('Saturday')

fdf['Weekday Name'] = weekdayn

monthn = []

for e in fdf['Month']:
    if e == 1:
        monthn.append('January')
    elif e == 2:
        monthn.append('Febuary')
    elif e == 3:
        monthn.append('March')
    elif e == 4:
        monthn.append('April')
    elif e == 5:
        monthn.append('May')
    elif e == 6:
        monthn.append('June')
    elif e == 7:
        monthn.append('July')
    elif e == 8:
        monthn.append('August')
    elif e == 9:
        monthn.append('September')
    elif e == 10:
        monthn.append('October')
    elif e == 11:
        monthn.append('November')
    else:
        monthn.append('December')

fdf['Monthn'] = monthn

Timedf = fdf.copy()
Timedf['Time'] = Timedf['Time'].str[:2]
Timedf['Time'] = Timedf['Time'].replace('0','00')


top10 = fdf.groupby(['Genre'])['Play Count'].sum().reset_index().sort_values('Play Count', ascending = False).head(10)['Genre'].tolist()

song50 = fdf.groupby(['Artist','Song'])['Play Count'].count().reset_index().sort_values('Play Count', ascending = True).head(50)['Song'].tolist()


In [57]:
fdf.head(1)

,Apple Music Subscription,Artist,Song,Content Provider,Content Specific Type,End Reason Type,Event Start Timestamp,Feature Name,Genre,Play Count,"""genre",Date,Year,Month,Weekday,Time,Weekday Name,Monthn
0,True,"""Weird Al"" Yankovic",Amish Paradise,Sony Music,Music Video,MANUALLY_SELECTED_PLAYBACK_OF_A_DIFF_ITEM,2018-07-11T05:19:36.558Z,browse,Comedy,1,NaN,2018-07-11 05:19:36.558000+00:00,2018,7,2,05:19,Wednesday,July


In [58]:
pdf = fdf[fdf['Genre']=='Hip Hop/Rap']
pdf = pdf.groupby(['Artist', 'Genre'])['Play Count'].count().reset_index().sort_values('Play Count', ascending = False)
top7 = pdf.groupby(['Artist'])['Play Count'].sum().reset_index().sort_values('Play Count', ascending = False).head(7)['Artist'].tolist()
pdf = pdf[pdf['Artist'].isin(top7)]

pdf.iloc[0,0]

'Kanye West'

In [59]:
            pdf = fdf[fdf['Genre'] == 'Rock']
            pdf = pdf[pdf['Year'] == 2018]
            pdf = pdf.groupby(['Month','Artist', 'Genre', 'Year', 'Monthn'])['Play Count'].count().reset_index().sort_values('Month', ascending = True)
            top7 = pdf.groupby(['Artist'])['Play Count'].sum().reset_index().sort_values('Play Count', ascending = False).head(7)['Artist'].tolist()
            pdf = pdf[pdf['Artist'].isin(top7)]
            pdf.head(50)

,Month,Artist,Genre,Year,Monthn,Play Count
15,1,Pink Floyd,Rock,2018,January,2
12,1,Meat Loaf,Rock,2018,January,10
8,1,Foreigner,Rock,2018,January,34
6,1,Dire Straits,Rock,2018,January,33
2,1,Black Label Society,Rock,2018,January,38
65,2,Tenacious D,Rock,2018,Febuary,3
55,2,Pink Floyd,Rock,2018,Febuary,10
50,2,Meat Loaf,Rock,2018,Febuary,6
23,2,Black Label Society,Rock,2018,Febuary,8
32,2,Dire Straits,Rock,2018,Febuary,12


In [60]:
fig = px.line(
    pdf,
    x = 'Month',
    y = 'Play Count',
    color = 'Artist',
    custom_data= ['Artist', 'Genre', 'Year', 'Monthn', 'Play Count']
)

fig.update_traces(
    hovertemplate = '<br>'.join([
        'Artist: <b>%{customdata[0]}</b>',
        'Genre: <b>%{customdata[1]}</b>',
        'Month: <b>%{customdata[3]}</b>',
        'Year: <b>%{customdata[2]}</b>',
        'Play Count: <b>%{customdata[4]}</b>'
    ])
)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    coloraxis_showscale = False,
)
fig.update_yaxes(
    {'showgrid': False}
    )

fig.update_xaxes(
    ticktext = [
        'January',
        'February',
        'March',
        'April',
        'May',
        'June',
        'July',
        'August',
        'September',
        'October',
        'November',
        'December'
    ],
    tickvals = [
        '1',
        '2',
        '3',
        '4',
        '5',
        '6',
        '7',
        '8',
        '9',
        '10',
        '11',
        '12'
    ]
)

fig.show()